Qwen3 (0.6B) 모델을 사용하여, [실습 1: LLM을 함수처럼 사용하기]와 [실습 2: 문서 기반 전문가(On-Device RAG)] 진행

In [1]:
import os
from langchain_ollama import ChatOllama
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [2]:
# Qwen3는 추론 능력이 강화되어 작은 사이즈로도 똑똑한 대답이 가능합니다.
# temperature=0: 창의성을 0으로 낮추어, 항상 일관되고 사실적인 답변만 하도록 설정 (데이터 추출에 필수)
llm = ChatOllama(model="qwen3:0.6b", temperature=0)

[실습 1] LLM을 함수처럼 사용하기

In [3]:
# Few-shot 프롬프트 (예시를 주입)
# 모델에게 "여러 개가 나오면 리스트([])로 묶어라"는 것을 예시로 학습시킵니다.
template = """
You are a precise Data Extraction Module.
Extract hardware status from the input text into a JSON List format.

Examples:
Input: "Battery is 12V and Camera is OK."
Output: [
    {{"component": "Battery", "status": "Normal", "value": 12}},
    {{"component": "Camera", "status": "OK", "value": null}}
]

Input: {input}
Output:
"""

prompt = ChatPromptTemplate.from_template(template)

In [4]:
chain = prompt | llm | StrOutputParser()

# 예: "지금 라이다 센서는 정상, 모터 온도 경고 상태"
user_input = "LiDAR 센서-> 정상 작동 중, 모터 온도가 45도라서 경고 상태야."

response = chain.invoke({"input": user_input})
print(response)

[
    {"component": "LiDAR sensor", "status": "Normal", "value": 12},
    {"component": "motor", "status": "Warning", "value": null}
]


**[실습 2]: 문서 기반 전문가(On-Device RAG)**

In [5]:
import numpy as np
from dotenv import load_dotenv
from google import genai
from google.genai import types
from pypdf import PdfReader

# 설정 및 API 클라이언트 초기화(임베딩 담당)
load_dotenv()
api_key = os.environ.get("GEMINI_API_KEY")
client = genai.Client(api_key=api_key)

유틸 함수

In [6]:
# 임베딩 및 검색 함수 정의 (Gemini의 '눈' 역할)
def get_embedding(text, task_type="RETRIEVAL_DOCUMENT"):
    response = client.models.embed_content(
        model="text-embedding-004",
        contents=text,
        config=types.EmbedContentConfig(task_type=task_type)
    )
    return response.embeddings[0].values

def search_pdf(query, chunks, chunk_embeddings):
    # 질문을 벡터로 변환
    query_vec = get_embedding(query, task_type="RETRIEVAL_QUERY")
    
    # 코사인 유사도 계산
    scores = [np.dot(query_vec, doc_vec) for doc_vec in chunk_embeddings]
    best_idx = np.argmax(scores)
    
    return chunks[best_idx]

메인 실행 로직

In [14]:
# PDF 읽기 및 벡터화 (데이터 준비)
pdf_path = "src/syllabus.pdf" 
print(f"PDF '{pdf_path}' 로딩 및 처리 중...")

reader = PdfReader(pdf_path)
chunks = [page.extract_text() for page in reader.pages if page.extract_text()]

# Gemini로 문서 전체 임베딩 (최초 1회 실행)
print(f"Vectorizing {len(chunks)} pages... (Gemini Embedding)")
chunk_embeddings = [get_embedding(chunk) for chunk in chunks]
print("벡터화 완료!\n")

Ignoring wrong pointing object 10 0 (offset 0)


PDF 'src/syllabus.pdf' 로딩 및 처리 중...
Vectorizing 5 pages... (Gemini Embedding)
벡터화 완료!



In [18]:
# 비교할 질문
test_question = "이번 모빌리티S/W활용II 과목의 교수 이름과 강의실, 그리고 이메일을 알려줘. 또 1일차 수업 내용을 요약해줘"

[Case 1] ❌ RAG 적용 전

In [16]:
print(f"--- [❌ RAG 적용 전 질문: {test_question}] ---")

template_no_rag = """
질문에 답변해 주세요.
질문: {question}
"""
prompt_no_rag = ChatPromptTemplate.from_template(template_no_rag)

# Chain 실행
chain_no_rag = prompt_no_rag | llm | StrOutputParser()
response_before = chain_no_rag.invoke({"question": test_question})

print(response_before)
print("-------------------------------------------\n")

--- [❌ RAG 적용 전 질문: 이번 모빌리티S/W활용II 과목의 교수 이름과 강의실, 그리고 강의 시간 및 요일을 알려줘.] ---
현재 진행 중인 "모빌리티 S/W 활용 II" 과목 관련 정보는 실시간으로 제공되지 않습니다. 아래와 같이 일반적인 정보를 알려드립니다:  
- **교수 이름**: [교수 이름 입력]  
- **강의실**: [강의실 번호/주소]  
- **강의 시간**: [강의 시간]  
- **요일**: [요일]  

이 정보가 실제 과목 내용을 바탕으로 제공되는지 확인해 주세요. 필요시 추가 정보를 알려주시면 더 구체적인 답변을 드릴 수 있습니다.
-------------------------------------------



[Case 2] ✅ RAG 적용 후 (검색 + 답변)

In [19]:
print(f"--- [RAG 적용 후 질문: {test_question}] ---")

# 1) 질문과 관련된 문서 내용(Context) 검색
found_context = search_pdf(test_question, chunks, chunk_embeddings)
print(f"🔍 [검색된 문서 내용 일부]:\n{found_context[:100]}...\n")

# 2) RAG 전용 프롬프트 정의
rag_template = """
You are a helpful assistant. Answer the question based ONLY on the context below.
If the answer is not in the context, say "I don't know".

Context:
{context}

Question:
{question}

Answer:
"""
rag_prompt = ChatPromptTemplate.from_template(rag_template)

# 3) Chain 실행 (Context와 Question을 함께 주입)
rag_chain = rag_prompt | llm | StrOutputParser()
response_after = rag_chain.invoke({
    "context": found_context,   # 검색한 내용
    "question": test_question   # 사용자 질문
})

print(f"Qwen({llm.model})의 RAG 답변:")
print(response_after)
print("-------------------------------------------")

--- [RAG 적용 후 질문: 이번 모빌리티S/W활용II 과목의 교수 이름과 강의실, 그리고 이메일을 알려줘. 또 1일차 수업 내용을 요약해줘] ---
🔍 [검색된 문서 내용 일부]:
수 업 계 획 서
( 2 0 2 5 학 년 도 겨 울 학 기 )
1 / 5
단 과 대 학 연 계 전 공 배 정 학 과 미 래 자 동 차 전 공
과 목 명 모 빌 리 티 S / W ...

Qwen(qwen3:0.6b)의 RAG 답변:
- **Teacher's Name**: 이 승 목  
- **Class Room**: 5 층 8 호 실  
- **Email**: s e u n g m o k @ k o o k m i n . a c . k r  
- **1-Day Class Summary**: 전 공 교 과 목 유 형 (Class: Public Education, Subject: Form)  

I don't know.
-------------------------------------------
